## Accessing SQLite Databases Using Python and Pandas
Objectives
* Access data stored in SQLite using Python.
* Use the sqlite3 module to interact with a SQL database.
* Use the pandas module to interact with a SQL database.

## Python and SQL
### The `sqlite3` module

In [ ]:
import sqlite3

# Create a SQL connection to our SQLite database
con = sqlite3.connect("doaj-article-sample.db")

cur = con.cursor()

# The result of a "cursor.execute" can be iterated over by row
for row in cur.execute('SELECT * FROM publishers'):
    print(row)

# Be sure to close the connection.
con.close()

### Queries

In [ ]:
# Create a SQL connection to our SQLite database
con = sqlite3.connect("doaj-article-sample.db")

cur = con.cursor()

In [ ]:
# Get all table names
query_str_all_tables = ' '.join([
    'SELECT name',
    'FROM sqlite_master',
    'WHERE type = "table"',
])
print(query_str_all_tables)

cur.execute(query_str_all_tables)
cur.fetchall()

In [ ]:
# Get information about the journals table
cur.execute('PRAGMA table_info(journals)')
cur.fetchall()

In [ ]:
# Get the top-10 journals with the highest average citation count
query_str = ' '.join([
    'SELECT ISSNs, AVG(Citation_Count)',
    'FROM articles',
    'GROUP BY ISSNs',
    'ORDER BY AVG(Citation_Count) DESC'
])
print(query_str)

cur.execute(query_str)
cur.fetchall()[:10]

In [ ]:
# Get all records where the Subject contains "Crystal Structure"
query_str = ' '.join([
    'SELECT Subjects',
    'FROM articles',
    'WHERE Subjects LIKE "%Crystal Structure%"'
])
print(query_str)

cur.execute(query_str)
results = cur.fetchall()

print(len(results), 'records.')
results[-5:]

In [ ]:
# Return only the first result of a query
query_str = ' '.join([
    'SELECT journals.Journal_Title, publishers.Publisher, COUNT(*)',
    'FROM articles',
    'JOIN journals',
    'ON articles.ISSNs = journals.ISSNs',
    'JOIN publishers',
    'ON publishers.id = journals.PublisherId',
    'GROUP BY Journal_Title',
    'ORDER BY COUNT(*) DESC'
])
print(query_str)

cur.execute(query_str)
cur.fetchone()

In [ ]:
# Be sure to close the connection.
con.close()

## Accessing data stored in SQLite using Python and Pandas

In [ ]:
import pandas as pd

# Read sqlite query results into a pandas DataFrame
con = sqlite3.connect("doaj-article-sample.db")
df = pd.read_sql_query("SELECT * FROM articles", con)
con.close()

# Verify that result of SQL query is stored in the dataframe
print(len(df), 'records.')
df.tail()

## Storing data: Create new tables using Pandas

In [ ]:
con = sqlite3.connect("doaj-article-sample.db")

# Load the data into a DataFrame
articles_df = pd.read_sql_query("SELECT * FROM articles", con)

# Select only articles published in November
articles_nov = articles_df[articles_df['Month'] == 11]
print(len(articles_nov), 'records in November.')

# Write the new DataFrame to a new SQLite table
articles_nov.to_sql("articles_nov", con, if_exists="replace")

# Get all table names with Pandas
tables = pd.read_sql_query(query_str_all_tables, con)

con.close()
tables